In [1]:
import requests
import pandas as pd
import urllib3
import time
from datetime import datetime
import os

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

## Login

In [2]:
def get_with_auth(url, login, password):
    # Define the parameters to send in the GET request
    params = {
        'username': login,
        'password': password
    }

    try:
        # Send GET request with parameters
        response = requests.get(url, params=params, verify=False ) #, cert=("C:/Users/FilipeFurlanBellotti/Documents/Vitrolife/API/API Prototype/cert/cert.pem", "C:/Users/FilipeFurlanBellotti/Documents/Vitrolife/API/API Prototype/cert/key.pem"))
        
        # Raise an exception if the request failed
        response.raise_for_status()
        
        # Return the response content
        return response  # Use response.json() for JSON data
    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL: {e}")
        return None

In [3]:
credentials = {
    'Vila Mariana':    {'ip': '10.250.115.39' , 'login': 'WEB', 'password': 'web'},
    'Ibirapuera':      {'ip': '10.250.110.208', 'login': 'WEB', 'password': 'web'},
    'Belo Horizonte':  {'ip': '192.168.215.43', 'login': 'WEB', 'password': 'web'},
    'Brasilia':        {'ip': '192.168.1.20'  , 'login': 'WEB', 'password': 'web'},
    'Salvador':        {'ip': '10.2.0.70'     , 'login': 'WEB', 'password': 'web'},
    # 'Ibirapuera II':   {'ip': '10.250.110.208', 'login': 'web', 'password': 'web'},
}


In [ ]:

for unidade in credentials.keys():
    print(f'staring unidade: {unidade}')
    
    try:
        base_url = f'https://{credentials[unidade]["ip"]}:4000/LOGIN'

        content = get_with_auth(base_url, credentials[unidade]['login'], credentials[unidade]['password'])

        token = content.json()['Token']

        print(f'unidade: {unidade} token: {token} \n')
    except Exception as e:
        print(f'unidade: {unidade} error: {e} \n')


staring unidade: Vila Mariana
unidade: Vila Mariana token: 95D455A499A0468DA8CBEACCE73FC928 

staring unidade: Ibirapuera
unidade: Ibirapuera token: 7985838E3A054DC5A14E4DAEE25615B2 

staring unidade: Belo Horizonte
unidade: Belo Horizonte token: 3F82DBE1E7C4471A852B8DF244138279 

staring unidade: Brasilia
unidade: Brasilia token: BB3BB4950BB7439D9466028489EE615B 

staring unidade: Salvador


In [7]:
base_url = f"https://{credentials['Belo Horizonte']['ip']}:4000/"
    
login_url = "LOGIN"

URL = base_url + login_url

content = get_with_auth(URL, 'Web', 'web')

token = content.json()['Token']

token

'CEA0C034F0BB4BD3AD2DD76B04B74129'

## Patients

In [8]:
def get_patients(base_url, token):
    # Define the URL for getting patients
    url = base_url + "/GET/patients"

    # Define the headers with the API token
    headers = {
        'API-token': token
    }

    try:
        # Send GET request to fetch patients
        response = requests.get(url, headers=headers, verify=False)
        
        # Raise an exception if the request failed
        response.raise_for_status()
        
        # Return the response content
        return response  # Assuming the response is in JSON format
    except requests.exceptions.RequestException as e:
        print(f"Error fetching patients: {e}")
        return None

url = base_url + "/GET/patients"


response = get_patients(base_url, token)

if response is not None:
    # Convert the response to a DataFrame
    patients = response.json()
    print(patients)
    df_patients = pd.DataFrame(patients['Patients'])



{'Patients': [{'PatientIDx': 'PC10T4JJ1843_43630.3958359954', 'PatientID': '40.438', 'FirstName': 'SIQUEIRA, LARA MARIA A B', 'LastName': '25/10/1986', 'DateOfBirth': '1986.10.01'}, {'PatientIDx': 'PC10T4JJ54678_43630.6286568634', 'PatientID': '44283', 'FirstName': 'CALDEIRA, JULIANA IMACULADA FERREIRA', 'LastName': '01/07/1982', 'DateOfBirth': '1982.07.01'}, {'PatientIDx': 'PC10T4JJ55173_43629.4403575000', 'PatientID': '54986', 'FirstName': 'Isabel Cristina da', 'LastName': 'Silva', 'DateOfBirth': '1979.07.01'}, {'PatientIDx': 'PC10T4JJ54678_43631.4704660995', 'PatientID': '50968', 'FirstName': 'PIRES, FABIANNE GOMES GASPAR BRANDAO', 'LastName': '16/11/1979', 'DateOfBirth': '1998.11.01'}, {'PatientIDx': 'PC10T4JJ54678_43631.4759475810', 'PatientID': '53102', 'FirstName': 'RODRIGUES, ENARA FERREIRA', 'LastName': '01/06/1985', 'DateOfBirth': '1985.06.01'}, {'PatientIDx': 'PC10T4JJ75642_43635.4462140162', 'PatientID': '57926', 'FirstName': 'SILVA, ALEXANDRA ALVARES E', 'LastName': '12/04

In [9]:
df_patients.head()

,PatientIDx,PatientID,FirstName,LastName,DateOfBirth
0,PC10T4JJ1843_43630.3958359954,40.438,"SIQUEIRA, LARA MARIA A B",25/10/1986,1986.10.01
1,PC10T4JJ54678_43630.6286568634,44283,"CALDEIRA, JULIANA IMACULADA FERREIRA",01/07/1982,1982.07.01
2,PC10T4JJ55173_43629.4403575000,54986,Isabel Cristina da,Silva,1979.07.01
3,PC10T4JJ54678_43631.4704660995,50968,"PIRES, FABIANNE GOMES GASPAR BRANDAO",16/11/1979,1998.11.01
4,PC10T4JJ54678_43631.4759475810,53102,"RODRIGUES, ENARA FERREIRA",01/06/1985,1985.06.01


# Treatments

In [10]:
def get_treatments(base_url, token, patientIDx):
    """
    Retrieve treatments for a given patientIDx from the API.

    Args:
        base_url (str): The base URL of the API.
        token (str): The API authentication token.
        patientIDx (str): The unique patient identifier.

    Returns:
        dict or None: The JSON response containing treatments, or None if request fails.
    """
    url = f"{base_url}/GET/TREATMENT"
    params = {'patientIDx': patientIDx}
    headers = {'API-token': token}

    try:
        response = requests.get(url, headers=headers, params=params, verify=False)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching treatments: {e}")
        return None

In [11]:
patient_idx = 'PC1PPEMH_45719.4711884144'
treatment_number = get_treatments(base_url, token, patient_idx)
print(treatment_number)

treatment_number = treatment_number['TreatmentList'][0]
treatment_number

{'TreatmentList': ['03/03/2025', '29/05/2025']}


'03/03/2025'

# Embryo ID

In [13]:
def get_embryo_id(base_url, token, patientIDx, treatment_name):
    """
    Retrieve embryo ID data for a given patientIDx and treatment name from the API.

    Args:
        base_url (str): The base URL of the API.
        token (str): The API authentication token.
        patientIDx (str): The unique patient identifier.
        treatment_name (str): The treatment name.

    Returns:
        dict or None: The JSON response containing embryo ID data, or None if request fails.
    """
    url = f"{base_url}/GET/embryoID"
    params = {
        'PatientIDx': patientIDx,
        'TreatmentName': treatment_name
    }
    headers = {'API-token': token}

    try:
        response = requests.get(url, headers=headers, params=params, verify=False)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching embryo ID: {e}")
        return None

In [14]:
get_embryo_id(base_url, token, patient_idx, treatment_number)

{'EmbryoIDList': ['D2025.03.03_S02823_I3254_P-1',
  'D2025.03.03_S02823_I3254_P-2']}

In [15]:
embryo_idx = get_embryo_id(base_url, token, patient_idx, treatment_number)['EmbryoIDList'][0]
embryo_idx

'D2025.03.03_S02823_I3254_P-1'

# Fertilization time

In [ ]:
def get_fertilization_time(base_url, token, embryo_id):
    """
    Retrieve fertilization time for a given EmbryoID from the API.

    Args:
        base_url (str): The base URL of the API.
        token (str): The API authentication token.
        embryo_id (str): The EmbryoID.

    Returns:
        dict or None: The JSON response containing fertilization time, or None if request fails.
    """
    url = f"{base_url}/GET/fertilizationtime"
    params = {'EmbryoID': embryo_id}
    headers = {'API-token': token}

    try:
        response = requests.get(url, headers=headers, params=params, verify=False)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching fertilization time: {e}")
        return None


In [ ]:
# Example usage:
get_fertilization_time(base_url, token, embryo_idx)

# Image runs

In [ ]:
def get_image_runs(base_url, token, embryo_id):
    """
    Retrieve image runs for a given EmbryoID from the API.

    Args:
        base_url (str): The base URL of the API.
        token (str): The API authentication token.
        embryo_id (str): The EmbryoID.

    Returns:
        dict or None: The JSON response containing image runs, or None if request fails.
    """
    url = f"{base_url}/GET/imageruns"
    params = {'EmbryoID': embryo_id}
    headers = {'API-token': token}

    try:
        response = requests.get(url, headers=headers, params=params, verify=False)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching image runs: {e}")
        return None


In [ ]:

# Example usage:
get_image_runs(base_url, token, embryo_idx)

# Evaluation - KID score

In [ ]:
def get_evaluation(base_url, token, embryo_id):
    """
    Retrieve evaluation data for a given EmbryoID from the API.

    Args:
        base_url (str): The base URL of the API.
        token (str): The API authentication token.
        embryo_id (str): The EmbryoID.

    Returns:
        dict or None: The JSON response containing evaluation data, or None if request fails.
    """
    url = f"{base_url}/GET/evaluation"
    params = {'EmbryoID': embryo_id}
    headers = {'API-token': token}

    try:
        response = requests.get(url, headers=headers, params=params, verify=False)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching evaluation: {e}")
        return None

In [ ]:
get_evaluation(base_url, token, embryo_idx)

# Emrbyo Fate

In [ ]:
def get_embryo_fate(base_url, token, embryo_id):
    """
    Retrieve embryo fate data for a given EmbryoID from the API.

    Args:
        base_url (str): The base URL of the API.
        token (str): The API authentication token.
        embryo_id (str): The EmbryoID.

    Returns:
        dict or None: The JSON response containing embryo fate data, or None if request fails.
    """
    url = f"{base_url}/GET/embryofate"
    params = {'EmbryoID': embryo_id}
    headers = {'API-token': token}

    try:
        response = requests.get(url, headers=headers, params=params, verify=False)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching embryo fate: {e}")
        return None


In [ ]:

# Example usage:
get_embryo_fate(base_url, token, embryo_idx)

# Embryo Details

In [ ]:
def get_embryo_details(base_url, token, embryo_id):
    """
    Retrieve detailed embryo information for a given EmbryoID from the API.

    Args:
        base_url (str): The base URL of the API.
        token (str): The API authentication token.
        embryo_id (str): The EmbryoID.

    Returns:
        dict or None: The JSON response containing embryo details, or None if request fails.
    """
    url = f"{base_url}/GET/embryodetails"
    params = {'EmbryoID': embryo_id}
    headers = {'API-token': token}

    try:
        response = requests.get(url, headers=headers, params=params, verify=False)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching embryo details: {e}")
        return None


In [ ]:

# Example usage:
get_embryo_details(base_url, token, embryo_idx)

# Embryo Data

In [16]:
def get_embryo_data(base_url, token, patient_idx, treatment_name):
    """
    Retrieve embryo data for a given PatientIDx and TreatmentName from the API.

    Args:
        base_url (str): The base URL of the API.
        token (str): The API authentication token.
        patient_idx (str): The unique patient identifier.
        treatment_name (str): The treatment name.

    Returns:
        dict or None: The JSON response containing embryo data, or None if request fails.
    """
    url = f"{base_url}/GET/embryodata"
    params = {
        'PatientIDx': patient_idx,
        'TreatmentName': treatment_name
    }
    headers = {'API-token': token}

    try:
        response = requests.get(url, headers=headers, params=params, verify=False)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching embryo data: {e}")
        return None

In [17]:



get_embryo_data(base_url, token, patient_idx, treatment_number)

{'EmbryoDataList': [{'EmbryoID': 'D2025.03.03_S02823_I3254_P-1',
   'EmbryoDetails': {'InstrumentNumber': 3254,
    'Position': 11,
    'WellNumber': 1,
    'FertilizationTime': '2025.03.03 11:45:00',
    'EmbryoFate': 'FrozenEmbryoTransfer',
    'EmbryoDescriptionID': 'AA1'},
   'AnnotationList': [{'Name': 'tPB2',
     'Time': 4.8,
     'Value': '45719.6915769907',
     'Timestamp': '2025.03.03 16:35:52'},
    {'Name': 'tPNa',
     'Time': 11.8,
     'Value': '45719.9831809954',
     'Timestamp': '2025.03.03 23:35:46'},
    {'Name': 'PN',
     'Time': 23.2,
     'Value': '2',
     'Timestamp': '2025.03.04 10:58:07'},
    {'Name': 'tPNf',
     'Time': 25.8,
     'Value': '45720.5631392361',
     'Timestamp': '2025.03.04 13:30:55'},
    {'Name': 't2',
     'Time': 28,
     'Value': '45720.6577872338',
     'Timestamp': '2025.03.04 15:47:12'},
    {'Name': 'Fragmentation',
     'Time': 37.8,
     'Value': '5',
     'Timestamp': '2025.03.05 01:34:26'},
    {'Name': 'MultiNucleation',
    

# Transfer

In [ ]:
def get_transfers(base_url, token, patient_idx):
    """
    Retrieve transfer data for a given PatientIDx from the API.

    Args:
        base_url (str): The base URL of the API.
        token (str): The API authentication token.
        patient_idx (str): The unique patient identifier.

    Returns:
        dict or None: The JSON response containing transfer data, or None if request fails.
    """
    url = f"{base_url}/GET/transfers"
    params = {'PatientIDx': patient_idx}
    headers = {'API-token': token}

    try:
        response = requests.get(url, headers=headers, params=params, verify=False)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching transfers: {e}")
        return None


In [ ]:

# Example usage:
get_transfers(base_url, token, patient_idx)

# IDA Score

In [ ]:
def get_idascore(base_url, token):
    """
    Retrieve IDASCORE data for a given EmbryoID from the API.

    Args:
        base_url (str): The base URL of the API.
        token (str): The API authentication token.
        embryo_id (str): The EmbryoID.

    Returns:
        dict or None: The JSON response containing IDASCORE data, or None if request fails.
    """
    url = f"{base_url}/GET/IDASCORE"
    headers = {'API-token': token}

    try:
        response = requests.get(url, headers=headers, verify=False)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching IDASCORE: {e}")
        return None

In [ ]:
# Call the function to get IDASCORE data
idascore_data = get_idascore(base_url, token)

# Check if data was retrieved successfully and 'Scores' key exists
if idascore_data and 'Scores' in idascore_data:
    # Convert the 'Scores' list into a DataFrame
    df_idascore = pd.DataFrame(idascore_data['Scores'])
else:
    print("No IDASCORE data found or 'Scores' key missing in the response.")
    df_idascore = pd.DataFrame() # Create an empty DataFrame if no data

df_idascore.head()

# Fluxo de um embryoscope

In [ ]:
def rate_limited_request(func, *args, **kwargs):
    """Execute function with rate limiting (10 requests per second)"""
    time.sleep(0.1)  # 100ms delay to stay under 10 requests/second
    return func(*args, **kwargs)

def get_all_treatments_for_patients(df_patients, base_url, token):
    """Get all treatments for all patients"""
    all_treatments = []
    
    print(f"Processing {len(df_patients)} patients...")
    
    for idx, patient in df_patients.iterrows():
        patient_idx = patient['PatientIDx']
        print(f"Processing patient {idx+1}/{len(df_patients)}: {patient_idx}")
        
        # Get treatments with rate limiting
        treatments_data = rate_limited_request(get_treatments, base_url, token, patient_idx)

        if treatments_data and 'TreatmentList' in treatments_data:
            for treatment_name in treatments_data['TreatmentList']:
                # Create treatment record with patient reference
                treatment_record = {
                    'PatientIDx': patient_idx,
                    'TreatmentName': treatment_name
                }
                all_treatments.append(treatment_record)
    
    df_treatments = pd.DataFrame(all_treatments)
    print(f"Found {len(df_treatments)} treatments across all patients")
    return df_treatments

def get_all_embryo_data(df_treatments, base_url, token):
    """Get embryo data for all treatments and break down nested dictionaries into columns"""
    all_embryo_data = []
    
    print(f"Processing {len(df_treatments)} treatments...")
    
    for idx, treatment in df_treatments.iterrows():
        patient_idx = treatment['PatientIDx']
        treatment_name = treatment['TreatmentName']
        
        print(f"Processing treatment {idx+1}/{len(df_treatments)}: {treatment_name}")
        
        # Get embryo data with rate limiting
        embryo_data = rate_limited_request(get_embryo_data, base_url, token, patient_idx, treatment_name)
        
        if embryo_data and 'EmbryoDataList' in embryo_data:
            for embryo in embryo_data['EmbryoDataList']:
                # Start with basic embryo info
                embryo_record = {
                    'EmbryoID': embryo['EmbryoID'],
                    'PatientIDx': patient_idx,
                    'TreatmentName': treatment_name,
                    'AnnotationList': embryo.get('AnnotationList')
                }
                
                # Break down EmbryoDetails into separate columns
                if 'EmbryoDetails' in embryo and embryo['EmbryoDetails']:
                    details = embryo['EmbryoDetails']
                    embryo_record.update({
                        'InstrumentNumber': details.get('InstrumentNumber'),
                        'Position': details.get('Position'),
                        'WellNumber': details.get('WellNumber'),
                        'FertilizationTime': details.get('FertilizationTime'),
                        'EmbryoFate': details.get('EmbryoFate'),
                        'Description': details.get('Description'),
                        'EmbryoDescriptionID': details.get('EmbryoDescriptionID')
                    })
                
                # Break down Evaluation into separate columns
                if 'Evaluation' in embryo and embryo['Evaluation']:
                    evaluation = embryo['Evaluation']
                    embryo_record.update({
                        'EvaluationModel': evaluation.get('Model'),
                        'EvaluationScore': evaluation.get('Evaluation'),
                        'EvaluationUser': evaluation.get('User'),
                        'EvaluationDate': evaluation.get('EvaluationDate')
                    })
                
                all_embryo_data.append(embryo_record)
    
    df_embryo_data = pd.DataFrame(all_embryo_data)
    print(f"Found {len(df_embryo_data)} embryos across all treatments")
    return df_embryo_data


In [ ]:
# Execute the flux
print("=== Starting Data Collection Flux ===")
print(f"Start time: {datetime.now()}")

# Step 1: We already have patients in df_patients
print(f"\nStep 1: Patients loaded - {len(df_patients)} patients")

df_patients.tail()

In [ ]:

# Step 2: Get all treatments
print("\nStep 2: Collecting treatments...")
df_treatments = get_all_treatments_for_patients(df_patients, base_url, token)

df_treatments.tail()

In [ ]:

# Step 3: Get all embryo data
print("\nStep 3: Collecting embryo data...")
df_embryo_data = get_all_embryo_data(df_treatments, base_url, token)

df_embryo_data.tail()

In [ ]:

print(f"\n=== Data Collection Complete ===")
print(f"End time: {datetime.now()}")
print(f"Total patients: {len(df_patients)}")
print(f"Total treatments: {len(df_treatments)}")
print(f"Total embryos: {len(df_embryo_data)}")

In [ ]:
# Merge df_patients with df_treatments on 'PatientIDx'
df_merged = pd.merge(df_patients, df_treatments, on='PatientIDx', how='left')

# Merge the result with df_embryo_data on 'PatientIDx' and 'TreatmentName'
df_final_merged = pd.merge(df_merged, df_embryo_data, on=['PatientIDx', 'TreatmentName'], how='left')

# Merge df_final_merged with df_idascore on 'EmbryoID'
df_final = pd.merge(df_final_merged, df_idascore, on='EmbryoID', how='inner')

print("Shape of df_patients:", df_patients.shape)
print("Shape of df_treatments:", df_treatments.shape)
print("Shape of df_embryo_data:", df_embryo_data.shape)
print("Shape of df_final_merged:", df_final_merged.shape)
print("Shape of df_idascore:", df_idascore.shape)
print("Shape of df_final:", df_final.shape)

df_final.tail()

In [ ]:
# Create data_output folder if it doesn't exist
os.makedirs('data_output', exist_ok=True)

# Save dataframes to CSV files
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Save without timestamp
df_patients.to_csv('data_output/patients.csv', index=False, quoting=1)
df_treatments.to_csv('data_output/treatments.csv', index=False, quoting=1)
df_embryo_data.to_csv('data_output/embryo_data.csv', index=False, quoting=1)
df_final.to_csv('data_output/final_merged_with_idascore.csv', index=False, quoting=1)

# Save with timestamp
df_patients.to_csv(f'data_output/patients_{timestamp}.csv', index=False, quoting=1)
df_treatments.to_csv(f'data_output/treatments_{timestamp}.csv', index=False, quoting=1)
df_embryo_data.to_csv(f'data_output/embryo_data_{timestamp}.csv', index=False, quoting=1)
df_final.to_csv(f'data_output/final_merged_with_idascore_{timestamp}.csv', index=False, quoting=1)

print(f"Files saved to data_output folder with timestamp: {timestamp}")

In [ ]:
# 1. Patients with no treatment
# Get unique PatientIDx from df_treatments (patients who have at least one treatment)
patients_with_treatment_ids = df_treatments['PatientIDx'].unique()
# Filter df_patients to find those whose PatientIDx is not in patients_with_treatment_ids
patients_without_treatment_df = df_patients[~df_patients['PatientIDx'].isin(patients_with_treatment_ids)]
num_patients_no_treatment = patients_without_treatment_df.shape[0]

# 2. Treatments with no embryo
# Create a marker for treatments that have at least one embryo
# Get unique (PatientIDx, TreatmentName) combinations from df_embryo_data
unique_treatments_in_embryo_data = df_embryo_data[['PatientIDx', 'TreatmentName']].drop_duplicates()
unique_treatments_in_embryo_data['has_embryo_marker'] = True

# Left merge df_treatments with the marked treatments from df_embryo_data
merged_treatments_check = pd.merge(
    df_treatments,
    unique_treatments_in_embryo_data,
    on=['PatientIDx', 'TreatmentName'],
    how='left'
)
# Treatments with no embryo will have NaN in 'has_embryo_marker'
num_treatments_no_embryo = merged_treatments_check['has_embryo_marker'].isnull().sum()

# 3. Embryos with no EvaluationScore AND no Viability
# Left merge df_embryo_data with selected columns from df_idascore to get Viability for each embryo
# This ensures all embryos from df_embryo_data are included
merged_embryos_for_score_check = pd.merge(
    df_embryo_data,
    df_idascore[['EmbryoID', 'Viability']],
    on='EmbryoID',
    how='left'
)

# Condition for missing EvaluationScore: either NaN/None or the string 'NA'
is_eval_score_missing = merged_embryos_for_score_check['EvaluationScore'].isnull() | \
                        (merged_embryos_for_score_check['EvaluationScore'] == 'NA')

# Condition for missing Viability: NaN (will occur if an embryo is not in df_idascore or if Viability was NaN there)
is_viability_missing = merged_embryos_for_score_check['Viability'].isnull()

# Filter for embryos missing both
embryos_missing_both_scores = merged_embryos_for_score_check[is_eval_score_missing & is_viability_missing]
num_embryos_no_eval_no_viability = embryos_missing_both_scores.shape[0]

# Print the report
print("Data Quality Report:")
print(f"- Patients with no treatment: {num_patients_no_treatment}")
print(f"- Treatments with no embryo: {num_treatments_no_embryo}")
print(f"- Embryos with no EvaluationScore AND no Viability: {num_embryos_no_eval_no_viability}")

In [ ]:
df_final.columns

In [ ]:
df_final['EmbryoFate'].value_counts(dropna=False)